In [1]:
import nest_asyncio
from playwright.async_api import async_playwright
import pandas as pd

nest_asyncio.apply()

async def scrape_all_products_async():
    all_products = []

    pw = await async_playwright().start()
    browser = await pw.chromium.launch(headless=False)
    page = await browser.new_page()

    # All methods are async (use the "await" keyword)
    await page.goto("https://web-scraping.dev/products")
    # src = await page.content()
    # print(src)

    while True:
        await page.wait_for_selector('div[class="products"]')
        cards = await page.query_selector_all('div[class="row product"]')

        for card in cards:
            # Thumbnail image
            thumbnail = await card.query_selector('div[class="col-2 thumbnail"]')
            img = await thumbnail.query_selector('img') if thumbnail else None

            # Description title and text
            description = await card.query_selector('div[class="col-8 description"]')
            title = await description.query_selector('h3') if description else None
            text = await description.query_selector('div[class="short-description"]') if description else None

            # Price
            price_wrap = await card.query_selector('div[class="col-2 price-wrap"]')
            price = await price_wrap.query_selector('div[class="price"]') if price_wrap else None
            
            all_products.append({
                'thumbnail': await img.get_attribute('src') if img else '',
                'title': await title.inner_text() if title else '',
                'description': await text.inner_text() if text else '',
                'price': await price.inner_text() if price else ''
            })

        paging = await page.query_selector('div[class="paging"]')
        next_page = (await paging.query_selector_all('a'))[-1] if paging else None
        if(next_page and await next_page.get_attribute('href')):
            await next_page.click()
            await page.wait_for_load_state('networkidle')  # or 'load'
        else:
            break

    await browser.close()
    await pw.stop()

    return pd.DataFrame(all_products)

# Run the async main function
df = await scrape_all_products_async()  # Use await directly instead of asyncio.run()
df.to_csv("products.csv", index=False)
df.head()

,thumbnail,title,description,price
0,https://web-scraping.dev/assets/products/orang...,Box of Chocolate Candy,Indulge your sweet tooth with our Box of Choco...,24.99
1,https://web-scraping.dev/assets/products/darkr...,Dark Red Energy Potion,Unleash the power within with our 'Dark Red Po...,4.99
2,https://web-scraping.dev/assets/products/teal-...,Teal Energy Potion,Experience a surge of vitality with our 'Teal ...,4.99
3,https://web-scraping.dev/assets/products/red-p...,Red Energy Potion,"Elevate your game with our 'Red Potion', an ex...",4.99
4,https://web-scraping.dev/assets/products/blue-...,Blue Energy Potion,Ignite your gaming sessions with our 'Blue Ene...,4.99
